In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


import os

import pickle
import random
from datetime import datetime
from typing import Dict, Tuple, Any
import gc
from tqdm import tqdm

import math

import numpy as np # linear algebra
np.set_printoptions(precision=6, suppress=True)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import QuantileTransformer

import cv2
import albumentations
from torch.utils.data import Dataset

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Dataset

import timm

from sklearn.preprocessing import normalize

In [3]:
BATCH_SIZE = 48
NUM_WORKERS = 0
EMBED_SIZE = 512
# 1/sqrt(2)
IMAGE_SIZE0 = int(np.round(600 * 0.70710677).item())
# 1
IMAGE_SIZE1 = 600
# sqrt(2)
IMAGE_SIZE2 = int(np.round(600 * 1.4142135).item())

DATA_DIR = '../input/'

MODEL_DIR = './model_checkpoints/'
MODEL_NAME = 'effnetv2m_in21k_fold2_epoch8.pth'

EMBEDDING_DIR = './embeddings'
TRAIN_EMBEDDING = 'effnetv2m_in21k_fold2_epoch8_embeddings.npy'
NONLM_EMBEDDING = 'effnetv2m_in21k_fold2_epoch8_test2019nonlandmark_embeddings.npy'
TRAIN_LABELS = 'trainfullfiltered_embeddings_targets.npy'

In [4]:
def cos_similarity_matrix(a, b, eps=1e-8):
    a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
    a_norm = a / torch.max(a_n, eps * torch.ones_like(a_n))
    b_norm = b / torch.max(b_n, eps * torch.ones_like(b_n))
    sim_mt = torch.mm(a_norm, b_norm.transpose(0, 1))
    return sim_mt

def get_topk_cossim(test_emb_in, tr_emb_in, batchsize=128, k=10, device='cuda:0', verbose=True):
    
    tr_emb = torch.tensor(tr_emb_in, dtype=torch.float16, device=torch.device(device))
    
    num_test_batches = np.ceil(test_emb_in.shape[0] / batchsize).astype('int')
    
    vals = []
    inds = []
    
    for i in tqdm(range(num_test_batches), disable=1-verbose):
        
        test_batch = torch.tensor(test_emb_in[i*batchsize : (i+1)*batchsize], dtype=torch.float16, device=torch.device(device))
        
        sim_mat = cos_similarity_matrix(test_batch, tr_emb)
        vals_batch, inds_batch = torch.topk(sim_mat, k=k, dim=1)
        
        vals += [vals_batch.detach().cpu()]
        inds += [inds_batch.detach().cpu()]
    
    vals = torch.cat(vals)
    inds = torch.cat(inds)
    
    del tr_emb, test_batch, vals_batch, inds_batch, sim_mat
    
    return vals, inds

def get_topk_cossim_sub(test_emb_in, tr_emb_in, vals_x_in, batchsize=128, k=10, device='cuda:0',verbose=True):
    
    tr_emb = torch.tensor(tr_emb_in, dtype=torch.float16, device=torch.device(device))
    vals_x = torch.tensor(vals_x_in, dtype=torch.float16, device=torch.device(device))
    
    num_test_batches = np.ceil(test_emb_in.shape[0] / batchsize).astype('int')
    
    vals = []
    inds = []
    
    for i in tqdm(range(num_test_batches), disable=1-verbose):
        
        test_batch = torch.tensor(test_emb_in[i*batchsize : (i+1)*batchsize], dtype=torch.float16, device=torch.device(device))
        
        sim_mat = cos_similarity_matrix(test_batch, tr_emb)
        sim_mat = torch.clamp(sim_mat, 0, 1) - vals_x.repeat(sim_mat.shape[0], 1)
        vals_batch, inds_batch = torch.topk(sim_mat, k=k, dim=1)
        
        vals += [vals_batch.detach().cpu()]
        inds += [inds_batch.detach().cpu()]
    
    vals = torch.cat(vals)
    inds = torch.cat(inds)
    
    del tr_emb, vals_x, test_batch, vals_batch, inds_batch, sim_mat
    
    return vals, inds

In [5]:
with open('../input/glr2021-idx2landmarks/idx2landmark_id.pkl', 'rb') as f:
    idx2lm = pickle.load(f)

load = torch.load(os.path.join(MODEL_DIR, MODEL_NAME))
arc_centers = load['model_state_dict']['module.metric_classify.weight'].cpu().numpy()
arc_centers = normalize(arc_centers)

del load
gc.collect()

embedding_train = np.load(os.path.join(EMBEDDING_DIR_NEW, TRAIN_EMBEDDING))

0

In [8]:
train_topK_similarities, train_topK_indices  = get_topk_cossim(arc_centers, embedding_train, k=10)

100%|██████████| 1906/1906 [05:53<00:00,  5.39it/s]


In [22]:
len(idx2lm)

81313

In [15]:
vals_nl[1]

tensor([0.2057, 0.2018, 0.1957, 0.1934, 0.1925, 0.1909, 0.1907, 0.1901, 0.1896,
        0.1885], dtype=torch.float16)

In [17]:
inds_nl[2]

tensor([ 262905, 2719220, 1656025, 1543719, 2193735, 1333143, 1366152, 2678412,
        1291678, 2076371])

In [ ]:
vals_nl.shape